In [3]:
import numpy as np
import pandas as pd
from copy import deepcopy
from tqdm import tqdm

In [4]:
txt='''inp w
mul x 0
add x z
mod x 26
div z 1
add x 14
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 8
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 1
add x 15
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 11
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 1
add x 13
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 2
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 26
add x -10
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 11
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 1
add x 14
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 1
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 26
add x -3
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 5
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 26
add x -14
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 10
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 1
add x 12
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 6
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 1
add x 14
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 1
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 1
add x 12
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 11
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 26
add x -6
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 9
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 26
add x -6
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 14
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 26
add x -2
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 11
mul y x
add z y
inp w
mul x 0
add x z
mod x 26
div z 26
add x -9
eql x w
eql x 0
mul y 0
add y 25
mul y x
add y 1
mul z y
mul y 0
add y w
add y 2
mul y x
add z y'''

In [5]:
program_str=[x.split(' ') for x in txt.split('\n')]

In [6]:


operator_signs={'inp':'d','mul':'*','add':'+','div':'/','mod':'%','eql':'=='}
class ALUProgram:
    def __init__(self,command,operator1=None,operator2=None):
        self.command=command
        self.simplified=False
        self.depends_on=set()
        self.input_dependant=False

        
        if operator1 is None:
            self.operator1=ALUProgram('cns',0)
        else:
            self.operator1=deepcopy(operator1)
            
                
        
        if operator2 is None:
            if command not in ['inp','cns']:
                self.operator2=ALUProgram('cns',0)
            else:
                self.operator2=None
        else:
            self.operator2=deepcopy(operator2)
        
        if (command=='inp'):
            self.input_dependant=True
            self.depends_on={operator1}
        elif (command=='cns'):
            self.input_dependant=False
        else:
            if type(operator1)==ALUProgram:
                self.input_dependant=operator1.input_dependant
                self.depends_on=operator1.depends_on
                
            if type(operator2)==ALUProgram:
                self.input_dependant=self.input_dependant or operator2.input_dependant
                self.depends_on=operator2.depends_on.union(self.depends_on)
                
                
            
    def is_operator1_input(self):
        return (type (self.operator1)==ALUProgram) and (self.operator1.command=='inp')
    
    def is_operator2_input(self):
        return (type (self.operator2)==ALUProgram) and (self.operator2.command=='inp')
    
    def is_operator1_cnst(self):
        return (type (self.operator1)==ALUProgram) and (self.operator1.command=='cns')
    
    def is_operator2_cnst(self):
        return (type (self.operator2)==ALUProgram) and (self.operator2.command=='cns')
    
    def copy_operator1(self):
        if type(self.operator1)==ALUProgram:
            self.input_dependant=False if self.operator1 is None else self.operator1.input_dependant
            self.depends_on=set() if self.operator1 is None else self.operator1.depends_on
            self.command=self.operator1.command
            anterior=self.operator1.operator2
            self.operator1=deepcopy(self.operator1.operator1)
            self.operator2=deepcopy(anterior)
            
        else:
            self.command='cns'
            self.operator1=self.operator1
            self.operator2=None
            self.input_dependant=False
            self.depends_on=set()
    
    def copy_operator2(self):
        if type(self.operator2)==ALUProgram:
            self.input_dependant=False if self.operator2 is None else self.operator2.input_dependant
            self.depends_on=set() if self.operator2 is None else self.operator2.depends_on
            self.command=self.operator2.command
            anterior=self.operator2.operator2
            self.operator1=deepcopy(self.operator2.operator1)
            self.operator2=deepcopy(anterior)
            
        else:
            self.command='cns'
            self.operator1=self.operator2
            self.operator2=None
            self.input_dependant=False
            self.depends_on=set()
            

    
    def simplificar(self,max_evaluation_d_deepth=6):
        
        if self.command not in ['cns','inp']:
            
            try:
                txt=str(self.__repr__())
                if not 'd' in txt:
                    valor=eval(txt)
                    if type(valor)==float:
                        valor=int(valor)
                    if type(valor)==bool:
                        valor=int(valor)
                    if type(valor)==int:
                        self.command='cns'
                        self.operator1=valor
                        self.operator2=None
                        self.input_dependant=False
                        self.depends_on=set()
            except: 
                print(f'error evaluant: {str(self.__repr__())}')
            
            if type(self.operator1)==ALUProgram:
                self.operator1.simplificar()
            if type(self.operator2)==ALUProgram:
                self.operator2.simplificar()

            if self.command=='eql':
                if self.is_operator1_input() and self.is_operator2_cnst():
                    if (self.operator2.operator1>9) or (self.operator2.operator1<0):
                        self.command='cns'
                        self.operator1=0
                        self.operator2=None
                        self.input_dependant=False
                        self.depends_on=set()

                elif self.is_operator2_input() and self.is_operator1_cnst():
                    if (self.operator1.operator1>9) or (self.operator1.operator1<0):
                        self.command='cns'
                        self.operator1=0
                        self.operator2=None
                        self.input_dependant=False
                        self.depends_on=set()
                
                        
            # multiplicacio per 1, multiplicacio per 0


            elif self.command=='mul':
                if (self.is_operator1_cnst()) and (self.operator1.operator1==0):
                        
                    self.command='cns'
                    self.operator1=0
                    self.operator2=None
                    self.input_dependant=False

                elif (self.is_operator2_cnst()) and (self.operator2.operator1==0):
                    self.command='cns'
                    self.operator1=0
                    self.operator2=None
                    self.input_dependant=False
                    self.depends_on=set()

                elif (type(self.operator2)==int) and (self.operator2==0):
                    self.command='cns'
                    self.operator1=0
                    self.operator2=None
                    self.input_dependant=False
                    self.depends_on=set()

                elif (type(self.operator1)==int) and (self.operator1==0):
                    self.command='cns'
                    self.operator1=0
                    self.operator2=None
                    self.input_dependant=False
                    self.depends_on=set()



                elif (self.is_operator1_cnst()) and (self.operator1.operator1==1):
                    self.copy_operator2()

                elif (self.is_operator2_cnst()) and (self.operator2.operator1==1):
                    self.copy_operator1()
                    
                elif (type(self.operator1)==int) and (self.operator1==1):
                    self.copy_operator2()

                elif (type(self.operator2)==int) and (self.operator2==1):
                    self.copy_operator1()


            elif self.command=='div':


                if (self.is_operator2_cnst()) and (self.operator1.operator1==1):
                        self.copy_operator1()


                elif (type(self.operator2)==int) and (self.operator2==1):
                        self.copy_operator1()


            elif self.command=='mod':

                if (self.is_operator2_cnst()) and (self.operator1.operator1==1):
                    self.command='cns'
                    self.operator1=0
                    self.operator2=None
                    self.input_dependant=False
                    self.depends_on=set()


                elif (type(self.operator2)==int) and (self.operator2==1):
                    self.command='cns'
                    self.operator1=0
                    self.operator2=None
                    self.input_dependant=False
                    self.depends_on=set()


            elif self.command=='add':

                if (self.is_operator1_cnst()) and (self.operator1.operator1==0):
                        self.copy_operator2()
                elif (self.is_operator2_cnst()) and (self.operator2.operator1==0):
                        self.copy_operator1()

                        
                elif (type(self.operator1)==int) and (self.operator1==0):
                        self.copy_operator2()

                elif (type(self.operator2)==int) and (self.operator2==0):
                        self.copy_operator1()


            if False and (max_evaluation_d_deepth>sum(self.depends_on)>0):
                n,v,_=self.compute_all_possible_value_outputs()
                if n==1:
                    self.command='cns'
                    self.operator1=int(v)
                    self.operator2=None
                    self.input_dependant=False
                    self.depends_on=set()
            
            self.simplified=True

    def compute_all_possible_value_outputs_old(self):
        
        
        variables=self.depends_on
        equation=str(self)
        values=0
        ident=''
        txt='values=[]\n'

        for i in variables:
            txt+=f'{ident}for d{i} in range(1,10):\n'
            ident+='\t'
        txt+=f'{ident}input=np.zeros(14)\n'
        for i in variables:
            txt+=f'{ident}input[{i}]=d{i}\n'
        txt+=f'{ident}values.append([input ,int({equation})])'
        exec(txt)
        n_outputs=len(np.unique([x[1] for x in values]))
        output=values[0][1]
        
        return n_outputs,output,values
    def compute_all_possible_value_outputs(self):
        
        
        variables=self.depends_on
        equation=str(self)
        rangos=[range(1,10) if i in variables else [0] for i in range(14)]
        values=[]
        v_input=np.zeros(14)
        for d0 in rangos[0]:
            v_input[0]=d0
            for d1 in rangos[1]:
                v_input[1]=d1
                for d2 in rangos[2]:
                    v_input[2]=d2
                    for d3 in rangos[3]:
                        v_input[3]=d3
                        for d4 in rangos[4]:
                            v_input[4]=d4
                            for d5 in rangos[5]:
                                v_input[5]=d5
                                for d6 in rangos[6]:
                                    v_input[6]=d6
                                    for d7 in rangos[7]:
                                        v_input[7]=d7
                                        for d8 in rangos[8]:
                                            v_input[8]=d8
                                            for d9 in rangos[9]:
                                                v_input[9]=d9
                                                for d10 in rangos[10]:
                                                    v_input[10]=d10
                                                    for d11 in rangos[11]:
                                                        v_input[11]=d11
                                                        for d12 in rangos[12]:
                                                            v_input[12]=d12
                                                            for d13 in rangos[13]:
                                                                v_input[13]=d13
                                                                
                                                                values.append([v_input.copy(),int(eval(equation))])
        n_outputs=len(np.unique([x[1] for x in values]))
        output=values[0][1]
        
        return n_outputs,output,values
        
                      
        
        
        
        
    def evaluar (self,input_vector):
        
        if self.command=='cns':
            ret_val=self.operator1
            
        elif self.command=='inp':
            ret_val=int(input_vector[self.operator1])
            
        elif self.command=='add':
            if type(self.operator2)==int:
                ret_val=self.operator1.evaluar()+self.operator2
            else:
                ret_val=self.operator1.evaluar()+self.operator2.evaluar()
        elif self.command=='mul':
            if type(self.operator2)==int:
                ret_val=self.operator1.evaluar()*self.operator2
            else:
                ret_val=self.operator1.evaluar()*self.operator2.evaluar()
        elif self.command=='div':
            if type(self.operator2)==int:
                ret_val=int(round(self.operator1.evaluar()/self.operator2))
            else:
                ret_val=int(round(self.operator1.evaluar()/self.operator2.evaluar()))
        elif self.command=='mod':
            if type(self.operator2)==int:
                ret_val=self.operator1.evaluar()%self.operator2
            else:
                ret_val=self.operator1.evaluar()%self.operator2.evaluar()
        elif self.command=='eql':
            if type(self.operator2)==int:
                ret_val=int(self.operator1.evaluar()==self.operator2)
            else:
                ret_val=int(self.operator1.evaluar()==self.operator2.evaluar())
           
                
                
        return ret_val

    def __repr__(self):
        if self.command=='inp':
            txt=f' d{self.operator1} '
        elif self.command=='cns':
            txt=str(self.operator1)
        
        elif self.command=='div':
            txt='( int ( round (' +str(self.operator1)+' '+operator_signs[self.command] +' '+str(self.operator2)+' ), 0) )'
        else:
            txt='  (' +str(self.operator1)+' '+operator_signs[self.command] +' '+str(self.operator2)+' )'
            
        return txt
        
    
class ALU():
    
    def __init__(self):
        self.xyzw=[ALUProgram('cns',0),ALUProgram('cns',0),ALUProgram('cns',0),ALUProgram('cns',0)]
        self.n_input=0
        
    def processar_line_code (self,command_str):
        register=list('xyzw').index(command_str[1])
        
        
        if command_str[0]=='inp':
            self.xyzw[register]=ALUProgram('inp',self.n_input)
            self.n_input+=1
        else:
            if command_str[2] in 'xyzw': ## operacion entre registros
                register2=list('xyzw').index(command_str[2])
                self.xyzw[register]=ALUProgram(command_str[0],self.xyzw[register],self.xyzw[register2])
                self.xyzw[register].simplificar()
                
            else:
                operator=int(command_str[2]) ## operacion con constante
                self.xyzw[register]=ALUProgram(command_str[0],self.xyzw[register],operator)
                self.xyzw[register].simplificar()
    @staticmethod
    def decode_text(txt):
        return [x.split(' ') for x in txt.split('\n')]
    
    def processar_programa(self,programa):
        
        for linea in tqdm(programa):
            
            self.processar_line_code(linea)
            
    def processar_programa_txt(self,txt):
        
        programa=self.decode_text(txt)
        for linea in programa:
            
            self.processar_line_code(linea)

                    
                
            
        
        
        

In [ ]:


my_ALU=ALU()
my_ALU.processar_programa(program_str)


 94%|█████████▍| 237/252 [1:00:14<1:09:57, 279.84s/it]